# Sorting Notebook

This notebook will download and sort electrophysiology collected using an Intan headstage, in the .rhd format. 

The data is intracranial mouse recording, from a 16 channel microarray. The paper can be found here: https://doi.org/10.1371/journal.pone.0221510


# Getting Set Up

Open a terminal. Make sure "Sorter" environment is active. 

```
conda deactivate
conda activate sorter
```

navigate to the correct directory, and examine the folders available. 

```
cd ~/codespace
box folders:items 352606395707
```

Download the patient level folder, which will contain multiple session folders. 
Replace with the correct file number.
```
box folders:download 123456789 --destination="data"
```

In [ ]:
# List the folders in the parent directory, copy the ID associated with the patient recording
!box folders:items 352606395707

----- Folder 352606396623 -----
Type: folder
ID: '352606396623'
Sequence ID: '0'
ETag: '0'
Name: Intan_RDH_2000

----- Folder 354522525287 -----
Type: folder
ID: '354522525287'
Sequence ID: '0'
ETag: '0'
Name: Intan_RDH_2000 (1)


In [2]:
!cd ~/codespace
!box folders:download 352606396623 --destination="data"

⠋ Starting download⠙ Starting download⠹ Starting download⠸ Starting download⠼ Creating folder 352606396623 at Intan_RDH_2000⠴ Creating folder 352606396623 at Intan_RDH_2000⠦ Creating folder 352605477299 at Intan_RDH_2000/Session1⠧ Creating folder 352605477299 at Intan_RDH_2000/Session1⠇ Creating folder 352605477299 at Intan_RDH_2000/Session1⠏ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠋ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠙ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠹ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠸ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠼ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠴ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd⠦ Downloading file 2054370143795 to Intan_RDH_2000/Session1/raw/Intan RHD fi

In [ ]:
import os
from pathlib import Path
from spikeinterface.sorters import run_sorter
import spikeinterface.full as si
import probeinterface as pi
from pathlib import Path
import probeinterface as pi


codespace = Path.home() / "codespace"
base_folder = codespace / "data"
patient = "Intan_RDH_2000"
session = "Session1"
session_location =  base_folder / patient / session
sorted_data = session_location / "sorted"
sorter_output_folder = sorted_data / "sorter_folder" 
analyzer_folder = sorted_data / "analyzer_folder"

os.chdir(session_location)

intan_file = "/home/marco/codespace/data/Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd"

# Load recording into spike interface

In [32]:
# Load Intan Recording
rec = si.read_intan(intan_file, stream_id = "0")
rec

IntanRecordingExtractor: 16 channels - 20.0kHz - 1 segments - 24,000,480 samples 
                         1,200.02s (20.00 minutes) - uint16 dtype - 732.44 MiB
  file_path: /home/marco/codespace/data/Intan_RDH_2000/Session1/raw/Intan RHD file1.rhd

In [ ]:
# # Create custom probe geometry

# probe = pi.Probe(ndim=2)
# positions = []

# for i in range(16):
#     positions.append([0, i * 50])
# probe.set_contacts(positions = positions, shapes = "circle", shape_params = {'radius':5})

# probe.set_device_channel_indices(range(16))
# probe.set_contact_ids([f"ch{i}" for i in range(16)])

# probe_path = codespace / "sorting_script/neuronexus-A16x1_2mm_50_177_A16.json"
# pi.write_probeinterface(probe_path, probe)


In [4]:
# Attach Probe to Recording
rec = rec.set_probe(probe)

n_rec = rec.get_num_channels()
n_probe = probe.get_contact_count()

if n_probe != n_rec:
    raise ValueError(f"Probe contacts ({n_probe}) != recording channels ({n_rec}). "
                     f"Pick the correct probe variant or subset/remap accordingly.")

In [ ]:
# Run Kilosort
from os import remove

sorting_KS4 = run_sorter(
    sorter_name="kilosort4",
    recording=rec,
    folder=sorter_output_folder,
    remove_existing_folder = True,
    verbose=True
)

write_binary_recording (no parallelization): 100%|██████████| 1201/1201 [00:08<00:00, 143.73it/s]
kilosort.run_kilosort:  
kilosort.run_kilosort: Computing preprocessing variables.
kilosort.run_kilosort: ----------------------------------------
kilosort.run_kilosort: N samples: 24000480
kilosort.run_kilosort: N seconds: 1200.024
kilosort.run_kilosort: N batches: 401
kilosort.run_kilosort: Preprocessing filters computed in 0.36s; total 0.36s
kilosort.run_kilosort:  
kilosort.run_kilosort: Resource usage after preprocessing
kilosort.run_kilosort: ********************************************************
kilosort.run_kilosort: CPU usage:     4.00 %
kilosort.run_kilosort: Mem used:      3.90 %     |       4.89 GB
kilosort.run_kilosort: Mem avail:    119.55 / 124.44 GB
kilosort.run_kilosort: ------------------------------------------------------
kilosort.run_kilosort: GPU usage:    `conda install pynvml` for GPU usage
kilosort.run_kilosort: GPU memory:    1.86 %     |      0.27   /    14.56 

kilosort4 run time 43.42s


In [ ]:
# Create Sorting Analyzer
import spikeinterface.full as si

# Load Recording
recording = si.read_intan(intan_file, stream_id = "0")
recording = recording.set_probe(probe, in_place=False)
recording = si.unsigned_to_signed(recording)
recording_filtered = si.bandpass_filter(recording)

job_kwargs = dict(n_jobs=-1, progress_bar=True, chunk_duration="1s")

sorting_analyzer = si.create_sorting_analyzer(sorting=sorting_KS4, recording=recording_filtered, folder=analyzer_folder, overwrite = True,
format="binary_folder", **job_kwargs)

sorting_analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
sorting_analyzer.compute("waveforms", **job_kwargs)
sorting_analyzer.compute("templates", **job_kwargs)
sorting_analyzer.compute("noise_levels")
sorting_analyzer.compute("unit_locations", method = "monopolar_triangulation")
sorting_analyzer.compute("isi_histograms")
sorting_analyzer.compute("correlograms", window_ms=100, bin_ms=5)
sorting_analyzer.compute("principal_components", n_components=3, mode="by_channel_global", whiten=True, **job_kwargs)
sorting_analyzer.compute("quality_metrics", metric_names=["snr", "firing_rate"])
sorting_analyzer.compute("template_similarity")
sorting_analyzer.compute("spike_amplitudes", **job_kwargs)

estimate_sparsity (workers: 32 processes): 100%|██████████| 1201/1201 [00:00<00:00, 2837.88it/s]
/home/marco/miniconda3/envs/sorter/lib/python3.11/site-packages/spikeinterface/core/basesorting.py:380: UserWarning: The registered recording will not be persistent on disk, but only available in memory
  warnings.warn("The registered recording will not be persistent on disk, but only available in memory")
spike_amplitudes (workers: 32 processes): 100%|██████████| 1201/1201 [00:01<00:00, 1140.97it/s]


In [12]:
analyzer_folder

PosixPath('/home/marco/codespace/data/Intan_RDH_2000/Session1/sorted/analyzer_folder')

# Run the curation window with the following code

sigui --mode=web --curation "/home/marco/codespace/data/Intan_RDH_2000/Session2/sorted/analyzer_folder"

In [ ]:
# Run Curation GUI

# I need to figure out how to clear the cache in between runs so that the data will be cleared, without getting phantom lables. 

# The reason is because the web app panel caches the curation json, probably as a way to try and be helpful. It needs to be flushed in between sessions. 
# import spikeinterface.full as si
# from spikeinterface_gui import run_mainwindow

# analyzer_folder = r"/home/marco/codespace/data/Intan_RDH_2000/Session2/sorted/analyzer_folder"

# sorting_analyzer = si.load_sorting_analyzer(folder=analyzer_folder)

# run_mainwindow(sorting_analyzer, mode="web", curation=True)

Found available port: 57861
Launching server at http://localhost:57861


In [ ]:
# Identify the client by Name, and copy the ID
!box folders:items 352606395707

----- Folder 352606396623 -----
Type: folder
ID: '352606396623'
Sequence ID: '0'
ETag: '0'
Name: Intan_RDH_2000

----- Folder 354522525287 -----
Type: folder
ID: '354522525287'
Sequence ID: '0'
ETag: '0'
Name: Intan_RDH_2000 (1)


In [ ]:
# List the folders inside of the client folder, each represents a session
# Copy the client ID from above to list the sessions
# Identify the correct session, and copy the ID
!box folders:items 352606396623

----- Folder 352605477299 -----
Type: folder
ID: '352605477299'
Sequence ID: '0'
ETag: '0'
Name: Session1

----- Folder 352604968054 -----
Type: folder
ID: '352604968054'
Sequence ID: '0'
ETag: '0'
Name: Session2


In [ ]:
# Upload the sorted folder to the correct session folder, using the ID copied from above
!box folders:upload "~/codespace/data/Intan_RDH_2000/Session1/sorted" --parent-folder 352605477299

Type: folder
ID: '354623627238'
Sequence ID: '0'
ETag: '0'
Name: sorted
Created At: '2025-12-06T13:02:43-08:00'
Modified At: '2025-12-06T13:02:43-08:00'
Description: ''
Size: 0
Path Collection:
    Total Count: 4
    Entries:
        -
            Type: folder
            ID: '0'
            Sequence ID: null
            ETag: null
            Name: All Files
        -
            Type: folder
            ID: '352606395707'
            Sequence ID: '0'
            ETag: '0'
            Name: Cloud_Sorter
        -
            Type: folder
            ID: '352606396623'
            Sequence ID: '0'
            ETag: '0'
            Name: Intan_RDH_2000
        -
            Type: folder
            ID: '352605477299'
            Sequence ID: '0'
            ETag: '0'
            Name: Session1
Created By:
    Type: user
    ID: '44567312476'
    Name: Marc Del Fava
    Login: mdelfava@ucdavis.edu
Modified By:
    Type: user
    ID: '44567312476'
    Name: Marc Del Fava
    Login: mdelfa

In [ ]:
# Upload the sorted folder to the correct session folder, using the ID copied from above
!box folders:upload "/home/marco/codespace/data/Intan_RDH_2000/Session2/sorted" --parent-folder 352604968054

Type: folder
ID: '354623985218'
Sequence ID: '0'
ETag: '0'
Name: sorted
Created At: '2025-12-06T13:06:28-08:00'
Modified At: '2025-12-06T13:06:28-08:00'
Description: ''
Size: 0
Path Collection:
    Total Count: 4
    Entries:
        -
            Type: folder
            ID: '0'
            Sequence ID: null
            ETag: null
            Name: All Files
        -
            Type: folder
            ID: '352606395707'
            Sequence ID: '0'
            ETag: '0'
            Name: Cloud_Sorter
        -
            Type: folder
            ID: '352606396623'
            Sequence ID: '0'
            ETag: '0'
            Name: Intan_RDH_2000
        -
            Type: folder
            ID: '352604968054'
            Sequence ID: '0'
            ETag: '0'
            Name: Session2
Created By:
    Type: user
    ID: '44567312476'
    Name: Marc Del Fava
    Login: mdelfava@ucdavis.edu
Modified By:
    Type: user
    ID: '44567312476'
    Name: Marc Del Fava
    Login: mdelfa